In [152]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import config

In [186]:
engine = create_engine(f"postgresql+psycopg2://{config.pguser}:{config.pgpassword}@{config.pghost}:{config.pgport}/{config.pgdatabase}")

connection = engine.connect()

# Get the database
sql = f'SELECT * FROM "matchdata"'
matchDate = pd.read_sql_query(sql,connection)

connection.close()

# matchDate = pd.read_csv(os.path.join('.', 'Resources', 'Speed Dating Preprocessed.csv'))
matchDate.head()

,couple_id,f_age,f_imprace,f_attr,f_sinc,f_intel,f_fun,f_amb,f_race,f_intrace,m_age,m_imprace,m_attr,m_sinc,m_intel,m_fun,m_amb,m_race,samerace,match,m_intrace
0,f1-m11,21.0,2.0,6.0,9.0,7.0,7.0,6.0,4.0,0.0,27.0,7.0,6.0,8.0,8.0,8.0,8.0,2.0,0,0,0.0
1,f1-m12,21.0,2.0,7.0,8.0,7.0,8.0,5.0,4.0,0.0,22.0,1.0,7.0,8.0,10.0,7.0,7.0,2.0,0,0,0.0
2,f1-m13,21.0,2.0,5.0,8.0,9.0,8.0,5.0,4.0,2.0,22.0,3.0,10.0,10.0,10.0,10.0,10.0,4.0,1,1,3.0
3,f1-m14,21.0,2.0,7.0,6.0,8.0,7.0,6.0,4.0,0.0,23.0,1.0,7.0,8.0,9.0,8.0,9.0,2.0,0,1,0.0
4,f1-m15,21.0,2.0,5.0,6.0,7.0,7.0,6.0,4.0,0.0,24.0,3.0,8.0,7.0,9.0,6.0,9.0,3.0,0,1,0.0


In [261]:
X_data = matchDate.drop(["couple_id","f_intrace","samerace", "match", "m_intrace"], axis=1)
X_data = X_data.round(decimals=0)
y_data = matchDate["match"]
print(X_data.shape, y_data.shape)

(3381, 16) (3381,)


Split our data into training and testing

In [260]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, random_state=1, stratify=y)

In [236]:
from sklearn.linear_model import LogisticRegression
lmodel = LogisticRegression(solver='newton-cg')
lmodel = lmodel.fit(X_train, y_train)
lmodel.score(X_test, y_test)

0.8392434988179669

Fit (train) or model using the training data

In [237]:
lpredictions = lmodel.predict(X_test)
from sklearn.metrics import confusion_matrix
tn, fp, fn, tp = confusion_matrix(y_test, lpredictions).ravel()
print(f"tn: {tn:3d}   fp: {fp:3d}")
print(f"fn: {fn:3d}   tp: {tp:3d}")

tn: 676   fp:  24
fn: 112   tp:  34


In [160]:
from sklearn.metrics import classification_report
print(classification_report(y_test, lpredictions, target_names=["0", "1"]))

              precision    recall  f1-score   support

           0       0.86      0.97      0.91       700
           1       0.59      0.23      0.33       146

   micro avg       0.84      0.84      0.84       846
   macro avg       0.72      0.60      0.62       846
weighted avg       0.81      0.84      0.81       846



In [238]:
from sklearn.ensemble import RandomForestClassifier
rfmodel = RandomForestClassifier(n_estimators=200)
rfmodel = rfmodel.fit(X_train, y_train)
rfmodel.score(X_test, y_test)

0.8439716312056738

Make predictions

In [239]:
rfpredictions = rfmodel.predict(X_test)
from sklearn.metrics import confusion_matrix
tn, fp, fn, tp = confusion_matrix(y_test, rfpredictions).ravel()
print(f"tn: {tn:3d}   fp: {fp:3d}")
print(f"fn: {fn:3d}   tp: {tp:3d}")


tn: 678   fp:  22
fn: 110   tp:  36


In [169]:
from sklearn.metrics import classification_report
print(classification_report(y_test, rfpredictions, target_names=["0", "1"]))

              precision    recall  f1-score   support

           0       0.86      0.96      0.91       700
           1       0.56      0.24      0.33       146

   micro avg       0.84      0.84      0.84       846
   macro avg       0.71      0.60      0.62       846
weighted avg       0.81      0.84      0.81       846



In [246]:
curr_user_profile = {'iid': [1],
                     'firstname': ['John'],
                     'lastname': ['Doe'],
                     'email': ['temp@unc.edu'],
                     'password': ['temp'],
                     'photo': [''],
                     'gender': [1],
                     'age': [22],
                     'race': [1],
                     'imprace': [2],
                     'attr': [8],
                     'sinc': [9],
                     'fun': [9],
                     'intel': [9],
                     'amb': [9]  }

my_df = pd.DataFrame(curr_user_profile)


engine = create_engine(f"postgresql+psycopg2://{config.pguser}:{config.pgpassword}@{config.pghost}:{config.pgport}/{config.pgdatabase}")

connection = engine.connect()

# Get the database
user_sql = f'SELECT * FROM "Users"'
df = pd.read_sql_query(user_sql,connection)

if (my_df["gender"][0] == 1):
    opp_df = df[df["gender"] == 0] 
    my_df["merge_gender"] = 0 
else:
   opp_df = df[df["gender"] == 1]  
   my_df["merge_gender"] = 1 

merge_df = pd.merge(opp_df, my_df, left_on='gender', right_on='merge_gender', how='left', suffixes=('_partner', '_user'))

pd.set_option('display.max_columns', None)
merge_df = merge_df.sort_values(by=['iid_partner'])

merge_df.head()




,iid_partner,firstname_partner,lastname_partner,password_partner,gender_partner,age_partner,screenname,email_partner,photo_partner,race_partner,imprace_partner,attr_partner,sinc_partner,intel_partner,fun_partner,amb_partner,iid_user,firstname_user,lastname_user,email_user,password_user,photo_user,gender_user,age_user,race_user,imprace_user,attr_user,sinc_user,fun_user,intel_user,amb_user,merge_gender
1,1,Elodie,Christiaens,Christiaens94~$,0,21.0,Elodie197,elodie94@mm.com,https://uinames.com/api/photos/female/20.jpg,4,2.0,6.700000,7.400000,8.000000,7.200000,8.000000,1,John,Doe,temp@unc.edu,temp,,1,22,1,2,8,9,9,9,9,0
46,2,Vüsalə,Əlizadə,Əlizadə92}(,0,24.0,Vüsalə965,vüsalə92@mm.com,https://uinames.com/api/photos/female/11.jpg,2,2.0,7.700000,7.100000,7.900000,7.500000,7.500000,1,John,Doe,temp@unc.edu,temp,,1,22,1,2,8,9,9,9,9,0
105,3,Végvári,Nóra,Nóra88}),0,25.0,Végvári730,végvári_88@mm.com,https://uinames.com/api/photos/female/26.jpg,2,8.0,6.555556,6.777778,7.222222,6.222222,7.111111,1,John,Doe,temp@unc.edu,temp,,1,22,1,2,8,9,9,9,9,0
170,4,Laura,Caraciobanu,Caraciobanu87=},0,23.0,Laura274,laura-87@mm.com,https://uinames.com/api/photos/female/17.jpg,2,1.0,7.000000,7.100000,7.700000,7.500000,7.700000,1,John,Doe,temp@unc.edu,temp,,1,22,1,2,8,9,9,9,9,0
213,5,Andreea,Trifa,Trifa88%_,0,21.0,Andreea936,andreea-88@mm.com,https://uinames.com/api/photos/female/10.jpg,2,8.0,5.300000,7.700000,7.600000,7.200000,7.800000,1,John,Doe,temp@unc.edu,temp,,1,22,1,2,8,9,9,9,9,0


In [251]:
X_match = merge_df[["age_partner", "imprace_partner", "attr_partner", "sinc_partner", "intel_partner", 
              "fun_partner", "amb_partner", "race_partner", "age_user", "imprace_user", 
              "attr_user", "sinc_user", "intel_user", "fun_user", "amb_user", "race_user"]]

X_match = X_match.round(decimals=0)
X_match.head()


,age_partner,imprace_partner,attr_partner,sinc_partner,intel_partner,fun_partner,amb_partner,race_partner,age_user,imprace_user,attr_user,sinc_user,intel_user,fun_user,amb_user,race_user
1,21.0,2.0,7.0,7.0,8.0,7.0,8.0,4,22,2,8,9,9,9,9,1
46,24.0,2.0,8.0,7.0,8.0,8.0,8.0,2,22,2,8,9,9,9,9,1
105,25.0,8.0,7.0,7.0,7.0,6.0,7.0,2,22,2,8,9,9,9,9,1
170,23.0,1.0,7.0,7.0,8.0,8.0,8.0,2,22,2,8,9,9,9,9,1
213,21.0,8.0,5.0,8.0,8.0,7.0,8.0,2,22,2,8,9,9,9,9,1


In [258]:
matches = rfmodel.predict(X_match)

match_list = []

for i in range(len(predictions)):
     if matches[i] == 1:                    
            match_list.append(merge_df["iid_partner"][i])  
    
print(match_list)

[180, 1, 112, 114, 115, 116, 117, 134, 142, 143, 149, 175, 178, 196, 2, 204, 207, 211, 212, 213, 236, 24, 240, 252, 255, 263, 264, 265, 270, 3, 30, 305, 38, 323, 325, 331, 35, 350, 359, 383, 384, 388, 390, 392, 395, 416, 417, 434, 458, 466, 474, 478, 498, 509, 515, 518, 519, 520, 522, 523, 525, 527, 528, 56, 57, 6, 60, 76, 78, 82, 84, 86]
